## Question to SPARQL query generation

In this notebook, we leverage the langchain API to generate a SPARQL query from an input plain english question.

LangChain has a ready-to-use method to generate SQL query, here we'll need to adapt it for SPARQL.

In [13]:
from aikg.config.chat import Config
from langchain import HuggingFacePipeline, LLMChain, PromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [1]:
template = """
We have provided the ontology of our RDF knowledge graph in turtle format below:
-----------------
{context}
-----------------
Using this ontology, please convert the following question to a SPARQL query:
{question}
"""

ontology = '/tmp/ontology.nt'

In [ ]:

!curl https://www.pokemonkg.org/ontology/ontology.nt -o $ontology

In [15]:
from rdflib import Graph
schema_graph = Graph().parse(ontology, format='nt')
config = Config()

In [16]:

tokenizer = AutoTokenizer.from_pretrained(config.chat_model_id)
model = AutoModelForCausalLM.from_pretrained(config.chat_model_id)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=128)

Loading checkpoint shards: 100%|██████████| 39/39 [00:51<00:00,  1.31s/it]


In [17]:

prompt = PromptTemplate(template=template, input_variables=["context", "question"])
llm = HuggingFacePipeline(pipeline=pipe)

In [18]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [19]:
llm_chain.run(
    question="Name 5 water pokemons",
    context=schema_graph.serialize(format='turtle')
)

/data/cyril/.cache/poetry/virtualenvs/aikg-URVQdnEY-py3.10/lib/python3.10/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


: 

: 